In [2]:
# ! nvidia-smi
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import logging
from sklearn.metrics import classification_report, confusion_matrix
from time import time
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
import random


tf.autograph.set_verbosity(0, False)
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [3]:
class ModifiedHebbTrainer:
    def __init__(self, input_size: int, output_size: int, modified=False):
        self.weights = np.zeros((input_size, output_size))
        self.modified = modified

    def train(self, inputs, outputs):
        for i, o in zip(inputs, outputs):
            i = i.reshape(1, -1)
            o = o.reshape(1, -1)
            self.weights += i.T @ o
        # Modification
        if self.modified:
            np.fill_diagonal(self.weights, 5)
    
    def predict(self, i):
        i = i.reshape(1, -1)
        pred = (i @ self.weights).reshape(-1)
        pred[pred >= 0] = 1
        pred[pred < 0] = -1
        return pred